In [1]:
import numpy as np

In [2]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
def get_train_valid_loader(data_dir,
                           batch_size,
                           random_seed,
                           augment=False,
                           valid_size=0.2,
                           shuffle=True,
                           show_sample=False,
                           num_workers=1,
                           pin_memory=True):
    """
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - augment: whether to apply the data augmentation scheme
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - valid_size: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot 9x9 sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg

    normalize = transforms.Normalize((0.1307,), (0.3081,))  # MNIST

    # define transforms
    valid_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize
        ])
    if augment:
        train_transform = transforms.Compose([
            #transforms.RandomCrop(32, padding=4),
            #transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
            #transforms.ColorJitter(brightness=0.2, contrast=0.2),
            #transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            
            transforms.ToTensor(),
            normalize
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize
        ])

    # load the dataset
    train_dataset = datasets.MNIST(root=data_dir, train=True, 
                download=True, transform=train_transform)

    valid_dataset = datasets.MNIST(root=data_dir, train=True, 
                download=True, transform=valid_transform)

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle == True:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(train_dataset, 
                    batch_size=batch_size, sampler=train_sampler, 
                    num_workers=num_workers, pin_memory=pin_memory)

    valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                    batch_size=batch_size, sampler=valid_sampler, 
                    num_workers=num_workers, pin_memory=pin_memory)


    # visualize some images
    if show_sample:
        sample_loader = torch.utils.data.DataLoader(train_dataset, 
                                                    batch_size=9, 
                                                    shuffle=shuffle, 
                                                    num_workers=num_workers,
                                                    pin_memory=pin_memory)
        data_iter = iter(sample_loader)
        images, labels = data_iter.next()
        X = images.numpy()
        plot_images(X, labels)

    return (train_loader, valid_loader)

In [4]:
def get_test_loader(data_dir, 
                    batch_size,
                    shuffle=True,
                    num_workers=1,
                    pin_memory=True):
    """
    Utility function for loading and returning a multi-process 
    test iterator over the MNIST dataset.
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - shuffle: whether to shuffle the dataset after every epoch.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - data_loader: test set iterator.
    """
    normalize = transforms.Normalize((0.1307,), (0.3081,))  # MNIST

    # define transform
    transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])

    dataset = datasets.MNIST(root=data_dir, 
                               train=False, 
                               download=True,
                               transform=transform)

    data_loader = torch.utils.data.DataLoader(dataset, 
                                              batch_size=batch_size, 
                                              shuffle=shuffle, 
                                              num_workers=num_workers,
                                              pin_memory=pin_memory)

    return data_loader

In [5]:
train_loader, valid_loader = get_train_valid_loader('../mnist_data',
                           batch_size = 16,
                           random_seed = 100,
                           augment=True,
                           valid_size=1/6,
                           shuffle=True,
                           show_sample=False,
                           num_workers=1,
                           pin_memory=True)

In [6]:
test_loader = get_test_loader('../mnist_data', 
                    batch_size=10,
                    shuffle=True,
                    num_workers=1,
                    pin_memory=True)

In [7]:
loaders = {'train': train_loader,
          'test': test_loader,
          'val': valid_loader} 

In [8]:
import torch.nn as nn
import torch.nn.functional as F

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2),
                                  )
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 3, 1, 1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2),
                                  )
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, 3, 1, 1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2),
                                  )
        self.out = nn.Linear(64*3*3, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = torch.flatten(x, 1)
        x = self.out(x)
        
        return x

In [10]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=576, out_features=10, bias=True)
)


In [11]:
params = list(cnn.parameters())
print(len(params))
print(params[0].size()) 

8
torch.Size([16, 1, 3, 3])


In [12]:
input = torch.randn(1, 1, 28, 28)
out = cnn(input)
print(out)

tensor([[ 0.2049, -0.1541,  0.0332,  0.1620,  0.1373,  0.0107, -0.2840, -0.0079,
          0.0425,  0.1598]], grad_fn=<AddmmBackward0>)


### Loss Function and Optimizer

In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.0001)

In [14]:
from torch.autograd import Variable
num_epochs = 10


In [15]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)

In [16]:
from torch.autograd import Variable
num_epochs = 10
def train(num_epochs, cnn, loaders):
    
    cnn.train()
        
    # Train the model
    total_step = len(loaders['train'])
        
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            
        
            outputs = cnn(images)              
            loss = criterion(outputs, labels)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            
            if (i+1) % 1000 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                pass
            
       
        pass
    
    
    pass
train(num_epochs, cnn, loaders)

Epoch [1/10], Step [1000/3125], Loss: 0.7441
Epoch [1/10], Step [2000/3125], Loss: 0.2061
Epoch [1/10], Step [3000/3125], Loss: 0.0535
Epoch [2/10], Step [1000/3125], Loss: 0.1239
Epoch [2/10], Step [2000/3125], Loss: 0.2902
Epoch [2/10], Step [3000/3125], Loss: 0.0192
Epoch [3/10], Step [1000/3125], Loss: 0.2101
Epoch [3/10], Step [2000/3125], Loss: 0.0095
Epoch [3/10], Step [3000/3125], Loss: 0.0253
Epoch [4/10], Step [1000/3125], Loss: 0.0070
Epoch [4/10], Step [2000/3125], Loss: 0.0062
Epoch [4/10], Step [3000/3125], Loss: 0.0680
Epoch [5/10], Step [1000/3125], Loss: 0.0692
Epoch [5/10], Step [2000/3125], Loss: 0.1340
Epoch [5/10], Step [3000/3125], Loss: 0.1421
Epoch [6/10], Step [1000/3125], Loss: 0.0113
Epoch [6/10], Step [2000/3125], Loss: 0.0037
Epoch [6/10], Step [3000/3125], Loss: 0.0053
Epoch [7/10], Step [1000/3125], Loss: 0.0745
Epoch [7/10], Step [2000/3125], Loss: 0.0458
Epoch [7/10], Step [3000/3125], Loss: 0.0074
Epoch [8/10], Step [1000/3125], Loss: 0.0048
Epoch [8/1

In [17]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = cnn(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 99.14 %
